In [2]:
import json
import pandas as pd
from collections import defaultdict

target = './detection/dataset/train.json'
category = {}
file_info = {}
make_frame = defaultdict(list)

# target 에 존재하는 train.json 파일을 엽니다.
with open(target, 'r') as f:
    json_datas = json.load(f) # python dict 처럼 접근하게끔 변환

    print(json_datas.keys())
    #dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

    # 이미지 정보 중 파일 경로와 아이디만 추출해서 file_info 에 저장
    for item in json_datas['images']:
        file_info[item['id']] = {'id' : item['id'], 'file_name' : item['file_name']}

    # 카테고리 정보를 category 에 저장
    for item in json_datas['categories']:
        category[item['id']] = item['name']

    # annotations 에 속하는 아이템들을 images 에 속하는 아이템의 정보와 합치기 위함
    for annotation in json_datas['annotations']:
        save_dict = file_info[annotation['image_id']]
        
        # 각 이미지에 해당하는 bounding box 정보와 class 정보 area(넓이) 정보를 추가
        save_dict.update({
            'class':annotation['category_id'],
            'x': annotation['bbox'][0],
            'y': annotation['bbox'][1],
            'w': annotation['bbox'][2],
            'h': annotation['bbox'][3],
            'area': annotation['area'],
            'ratio': annotation['bbox'][3] / annotation['bbox'][2]
            })

        for k,v in save_dict.items():
            # dataframe 으로 만들기 위해서 'key' : [item1,item2...] 형태로 저장
            make_frame[k].append(v)
    # print(make_frame)

    # dictionary 가 잘 만들어 졌는지 길이를 측정해서 확인해보세요!
    print(len(json_datas['annotations']))
    # dictionary to DataFrame
    a = pd.DataFrame.from_dict(make_frame)
    # index 를 제외하고 csv 파일로 저장
    a.to_csv('detection_info.csv',index=False)
    print(a)

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
23144
         id       file_name  class      x      y      w      h       area  \
0         0  train/0000.jpg      0  197.6  193.7  547.8  469.7  257301.66   
1         1  train/0001.jpg      3    0.0  407.4   57.6  180.6   10402.56   
2         1  train/0001.jpg      7    0.0  455.6  144.6  181.6   26259.36   
3         1  train/0001.jpg      4  722.3  313.4  274.3  251.9   69096.17   
4         1  train/0001.jpg      5  353.2  671.0  233.7  103.4   24164.58   
...     ...             ...    ...    ...    ...    ...    ...        ...   
23139  4882  train/4882.jpg      5    0.0  116.2  944.1  814.1  768591.81   
23140  4882  train/4882.jpg      7  302.1  439.3  265.2  216.1   57309.72   
23141  4882  train/4882.jpg      0  511.3  451.1   58.7   30.2    1772.74   
23142  4882  train/4882.jpg      1  255.0  421.4  271.7  195.1   53008.67   
23143  4882  train/4882.jpg      1  145.4  295.4  420.2  356.1  149633.22   

In [78]:
a['ratio'].describe()

count    23144.000000
mean         1.207635
std          0.986826
min          0.036649
25%          0.650489
50%          0.954637
75%          1.446871
max         18.053571
Name: ratio, dtype: float64

In [5]:
target = './detection/dataset/train.json'
category = {}
file_info = {}
make_frame = defaultdict(list)

# target 에 존재하는 train.json 파일을 엽니다.
with open(target, 'r') as f:
    json_datas = json.load(f) # python dict 처럼 접근하게끔 변환

    # 이미지 정보 중 파일 경로와 아이디만 추출해서 file_info 에 저장
    for item in json_datas['images']:
        file_info[item['id']] = {'id' : item['id'], 'file_name' : item['file_name']}

In [74]:
from tqdm import tqdm
from glob import glob
import os
import numpy as np
from PIL import Image

def get_img_stats(img_dir, img_ids):
    """
    데이터셋에 있는 이미지들의 크기와 RGB 평균 및 표준편차를 수집하는 함수입니다.
    
    Args:
        img_dir: 학습 데이터셋 이미지 폴더 경로 
        img_ids: 학습 데이터셋 하위폴더 이름들

    Returns:
        img_info: 이미지들의 정보 (크기, 평균, 표준편차)
    """
    img_info = dict(heights=[], widths=[], means=[], stds=[])
    for img_id in tqdm(img_ids):
        img = np.array(Image.open(os.path.join(img_dir, img_id)))
        h, w, _ = img.shape
        img_info['heights'].append(h)
        img_info['widths'].append(w)
        img_info['means'].append(img.mean(axis=(0,1)))
        img_info['stds'].append(img.std(axis=(0,1)))
    return img_info

In [75]:
FilePath = []

# print(file_info)

for file in file_info.values():
    FilePath.append(file['file_name'])
    
print(len(FilePath))
print(type(FilePath))

4883
<class 'list'>


In [76]:
img_dir = './detection/dataset'

In [77]:
img_info = get_img_stats(img_dir, FilePath)

100%|██████████| 4883/4883 [10:26<00:00,  7.80it/s]

RGB Mean: [0.48490459 0.46038158 0.43166834]
RGB Standard Deviation: [0.21190031 0.20929289 0.21483885]


In [80]:
print(f'RGB Mean: {np.mean(img_info["means"], axis=0)}')
print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0)}')

RGB Mean: [123.6506697  117.39730243 110.07542563]
RGB Standard Deviation: [54.03457934 53.36968771 54.78390763]


In [ ]:
import json